# Introduction : Modern Portfolio Theory

**Modern Portfolio Theory** est une théorie de l'investissement qui essaye de maximiser le rendement attendu sur la base d'un niveau de risque donné en choisissant minutieusement la proportions d'actifs dans le portefeuille.

Un portefeuille est un regroupement de différents types d'investissements tels que des actions, des obligations, des matières premières, des biens immobiliers, etc.

La théorie du portefeuille moderne suggère que le rendement attendu d'un investissement est directement lié au risque associé. Elle suggère qu'un investisseur doit choisir un portefeuille en fonction de son aversion au risque et de son horizon de placement.

Nous allons développer ici du code Python pour calculer les rendements attendus et la volatilité (risque) d'un portefeuille.

On utilise la bibliothèque pandas_datareader pour récupérer les données, numpy pour les opérations mathématiques, matplotlib pour visualiser et scipy pour l'optimisation.

Dans la suite de ce notebook, nous allons utiliser les notations suivantes :

- $r_p$ : rendement du portefeuille
- $w_i$ : proportion de l'actif $i$ dans le portefeuille
- $r_i$ : rendement de l'actif $i$
- $\sigma_i$ : volatilité de l'actif $i$
- $\sigma_p$ : volatilité du portefeuille

Notre objectif est de trouver les proportions $w_i$ qui maximisent le rendement du portefeuille pour un niveau de risque donné :

$$
\begin{align}
\max_{w_i} \quad & r_p \\
\text{sous contrainte} \quad & \sigma_p \leq \sigma_{max}
\end{align}
$$

In [ ]:
#pip install numpy pandas matplotlib scipy pandas_datareader plotly nbformat

In [5]:
# Importation des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas_datareader as web
import yfinance as yf
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_dark"

## Etape 1 : Récupération des données

Dans cette étape, nous allons récupérer les données des actifs qui composent notre portfolio. Pour cet exemple, nous allons considérer 3 actions : Google ('GOOG'), Apple ('AAPL') et Microsoft ('MSFT'). Mais vous pouvez ajouter autant d'actions que vous le souhaitez.

In [2]:
# Liste des actifs dans le portefeuille
assets = ['AAPL', 'GOOG', 'MSFT']

# Téléchargement des données
#data = web.DataReader(assets, data_source='yahoo', start='2010-01-01', end='2020-12-31')['Adj Close']
# use yfinance to fix pandas_datareader's issue
data = yf.download(assets, start='2010-01-01', end='2020-12-31')['Adj Close']

[*********************100%%**********************]  3 of 3 completed


In [9]:
# Affichage des données superposé avec plotly pour une année 
fig = go.Figure()
for a in assets:
    fig.add_trace(go.Scatter(x=data.index, y=data[a], name=a))
fig.update_layout(title='Evolution des prix des actifs', yaxis_title='Prix en $')
fig.show()

Comme nous avons vu dans le cours, pour faire de l'optimisation de portefeuille, nous avons besoin des rendements historiques des actifs. Nous allons donc récupérer les données sur une période de 5 ans.

## Etape 2 : Calcul des returns

Dans cette étape, nous allons calculer les rendements quotidiens des actions. Les rendements sont calculés en utilisant la formule suivante :
$$
r_t = \frac{P_{t} - P_{t-1}}{P_{t-1}}
$$
Où $P_{t}$ est le prix à la date $t$ et $P_{t-1}$ est le prix à la date $t-1$.
En fait, nous calculons le pourcentage de changement de prix entre deux jours consécutifs, c'est pour cela que l'on divise par $P_{t-1}$.

In [13]:
def compute_portfolio_return(data):
    return data.pct_change()

In [14]:
returns = compute_portfolio_return(data)
# Affichage des rendements journaliers
fig = go.Figure()
for a in assets:
    fig.add_trace(go.Scatter(x=returns.index, y=returns[a], name=a))
fig.update_layout(title='Rendements journaliers des actifs', yaxis_title='Rendements')
fig.show()

Maintenant, essayons de visualiser la distribution des rendements des actions.

In [12]:
# Returns distribution
fig = go.Figure()
for a in assets:
    fig.add_trace(go.Histogram(x=returns[a], name=a))
fig.update_layout(title='Distribution des rendements journaliers', yaxis_title='Fréquence')
fig.show()

## Etape 3 : Matrice de covariance